# Neural Turing Machine

* https://arxiv.org/pdf/1410.5401.pdf
* [The Moring Paper - Neural Turing Machine](https://blog.acolyer.org/2016/03/09/neural-turing-machines/)


## Pscyhology and Neuroscience 

Neural Turing Machine은 인간의 working memory와 매우 유사합니다. <br>
Working Memory는 short-term memory라고도 부르며, prefrontal cortex 그리고 basal ganglia로 이루어진 부분이며 많은 연구가 진행되었습니다. 대표적인 예로 [Monkey Working Memory Test](https://www.youtube.com/watch?v=zsXP8qeFF6A)가 있습니다. 

Working memory의 capacity limitation부분은 앞서 영상에서 보듯이 인간의 working memory는 매우 한정적입니다. <br>
그런 의미에서 short-term memory라고 하지만, Neural Turing Machine의 working memory는 인간처럼 한정적이지 않으며 [논문](https://arxiv.org/pdf/1410.5401.pdf)에 따르면 기존 LSTM (Long-Short Term Memory)보다 더 좋은 예측률을 보이고 있습니다.

## NTM Structure

Neural Turing Machine (NTM)은 2개의 components로 구성되어 있습니다. 

1. Neural Network Controller (그림에서 controller부분이고, NN으로 구성되어 있음)
2. Memory Bank


<img src="./images/neural-turing-machines.png">

Controller부분이 Neural Network로 구성되어 있으며, 보통의 NN과 마찬가지로 Input을 받으면 Output을 내놓습니다. <br>
일반적인 NN과 다른점은 Selective Read 그리고 Write를 사용해서 Memory Matrix와 interact합니다. <br>

궁극적으로 가장 중요한점은 모든 components들은 differentiable하다는 점 입니다. 즉. Gradient Descent로 학습이 가능합니다. <br>
Defferentiation 할수 있기 위해서, "Blurry" read 그리고 write operations를 통해 기존의 Turing Machine처럼 정확하게 하나하나 element를 다루는게 아니라, NN을 통해서 사람이 사고하듯이 메모리에 접근하도록 만들었습니다. 

<img src="./images/memory-block.png">

## Reading 

$ M_t $는 N * M memory metrix의 특정 시간 t의 메모리 정보입니다. <br>
N은 memory locations의 갯수를 가르키며, M은 각각 location의 vector size를 가르킵니다. <br>
$ w_t $ 는 특정시간 t의 N locations에 대한 a vector of weightings (가중치)입니다. <br>
모든 weightings(가중치)는 normalized되어 있기 때문에, N elements $ w^{(i)}_t = 1 $ 는 다음과 같은 조건을 갖습니다.<br>

> <span style="color:#777777;">Anderson: 아마도 $w_t$ 라는 것은 N에 해당하는 memory locations을 어디를 읽을지 가중치를 주는듯 하며, 이것을 NN을 통해서 구현하는것으로 보임</span>

$$ \sum_i w^{(i)}_t = 1 $$

$$ 0 \leq w^{(i)}_t \leq 1, \forall i $$

*the length $ M $ read vector* $ r_t $ 는 메모리안의 the row-vectors $ M^{(i)}_t $의 convex combination입니다. 

$$ r_t = \sum_i w^{(i)}_t M^{(i)}_t $$



## Writing

LSTM의 input 그리고 forget gates에 영향을 받아서, Neural Turing Machine또한 writing부분을 **erase** 그리고 **add** parts 2개로 나누었습니다. <br>

### Erase part

$$ \overset{\sim}{M}^{(i)}_t  = M^{(i)}_{t-1} \left[ 1 - w^{(i)}_t e_t  \right] $$

* $ w_t $: weighting 
* $ e_t $: erase vector
* $ M \in (0, 1) $: M의 모든 elements들은 0, 1사이에 존재

만약 특정 위치$ (i) $의 weighting vector $ w^{(i)}_t $ 와 erase vector  $ e_t $둘다 모두 **1값**을 갖고 있으면.. $ \left[1 - w^{(i)}_t e_t \right] $의 값이 0이 되면서 $ \overset{\sim}{M}^{(i)}_t $ 의 값은 0으로 됩니다.<br>
만약 weighting vector $ w^{(i)}_t $ 또는  erase vector  $ e_t $ 하나라도 **0값**을 갖고 있으면 .. 메모리는 변화되지 않습니다.

### Add part

Write head는 Erase part뿐만 아니라 Add part인 *length $ M $ add vector* $ a_t $또한 제공합니다. <br>
add part는 erase step이후에 메모리에 추가가 됩니다. 

$$ M^{(i)}_t = \overset{\sim}{M}^{(i)}_t + w^{(i)}_t a_t  $$

## Addressing Mechanisms


<img src="./images/addressing-mechanism.png">


1. $ k_t $: key vector
2. $ \beta_t $: key strength는 content-based addressing of the memory metrix $ M_t $를 연산하는데 사용
3. 결과값 content-based weighting은 *interpolation gate* $ g_t $에 기초한 이전 step의 weighting과 interpolation됨 
4. *shift weighting* $ s_t $는 얼마만큼의 weighting을 회전시킬지 결정합니다. 
5. 마지막으로, $ \gamma_t $에 따라서 weighting은 sharpened되고 최종적으로 memory access에 사용이 됩니다. 


### Content-based Addressing

weightings는 2개의 addressing mechanism을 complementary facilities와 합침으로서 생성이 됩니다. <br>
첫번째 mechanism인 **"content-based addressing"**는 current values와 controller에서 제공된 values의 유사성에 기반한 "위치"에 중점을 둡니다. (Hopfield network와 유사합니다.) 

content-based addressing의 장점은 retrieval 이 매우 단순합니다.<br>
즉 단순히 Controller는 저장된 데이터의 일부의 approximation을 만들고, 해당 approximation은 메모리와 비교하여 정확하게 저장된 값을 내놓도록 합니다. 

### Location-based Addressing

모든 문제가 content-based addressing으로 해결될수 있지 않습니다. <br>
예를 들어서 $ f(x, y) = x \cdot y $ 에서 $ x, y $는 **arbitrary** value를 갖을수 있습니다. <br>
Controller는 $ x, y $의 값을 메모리에 저장한뒤, 꺼내서 연산 $ x \cdot y $를 할 수 있습니다. <br>
이러한 경우에는 변수들 $x , y $의 addressing은 content가 아닌 location에 의해서 이루어집니다. 

### which one?

일반적으로 content-based address이 location-based addressing보다 더 많이 사용됩니다. <br>
이유는 content그 자체에서 location의 위치값을 찾을수 있기 때문입니다. 

### Focusing by Content  

Content-based addressing에서 head는 a length M *key vector* $ k_t $값을 제공하며, 해당 값은 similarity measure $ K[\cdot, \cdot]에 의해서 $ 각각의 vector $ M^{(i)}_t $와 비교됩니다. <br>
유사도와 positive key strength $ \beta_t $에 기초한 normalised weighting $ w^c_t $값을 연산할수 있습니다. <br>
$ \beta_t $ 값에 따라서 focus의 precision을 강화하거나 약하게 만들수 있습니다. 

$$ w^c_t(i) = \frac{exp\left( \beta_t K \left[ k_t, M^{(i)}_t \right] \right)}{\sum_j exp\left( \beta_t K \left[ k_t, M^{(j)}_t \right] \right)} $$

Similarity measure은 cosine similarity를 사용합니다. 

$$ K\left[ u, v \right] = \frac{ u \cdot v}{\| u \| \cdot \| v \|} $$

### Focusing by Location

Location-based addressing은 전체 메모리 위치에 대한 단순 iteration 또는 random-access jumps를 돕도록 만들어 져 있습니다. <br>
Rotational shift of a weighting을 사용함으로서 메모리를 찾습니다.<br>
예를 들어서 a rotation of 1은 focus를 다음 location으로 이동(shift)시킬 것 입니다.

Rotation을 하기 전에, 각각의 head는 scalar 값인 *interpolation gate* $ g_t \in (0, 1) $ 값을 내놓습니다. <br>
$ g $ 값은 head에 의해서 만들어진 weighting $ w_{t-1} $ 그리고 content system에 의해서 만들어진 weighting $ w^c_t $을 blend시키며 **gated weighting** $ w^g_t $를 결과값으로 내놓습니다. 

$$ w^g_t = g_t w^c_t + (1 - g_t) w_{t-1} $$

만약 gate $ g_t $값이 0이라면 content weighting $ w^c_t $값은 완전히 무시되고, 이전 step의 weighting 값이 쓰이게 될 것입니다. <br>
반대로 gate $ g_t $값이 1이라면 이전 iteration에서 나온 weighting값은 무시되고, content-based addressing이 적용이 됩니다. 